In [1]:
import os
import pandas as pd
import json
import duckdb
from datetime import datetime

## Look through the anova analysis results

In [4]:
parquet_file_key = [
    {"bias": "Bias_combined_anova_results_20250308_155958_v3.parquet"},
    {"ethics": "Ethics_combined_anova_results_20250308_160000_v3.parquet"},
    {"fairness": "Fairness_combined_anova_results_20250308_155959_v3.parquet"},
    {"factuality": "Factuality_combined_anova_results_20250308_160000_v3.parquet"}
]
anova_directory = "dataset/eda/anova/"

In [7]:

# Initialize DuckDB connection (in-memory)
con = duckdb.connect(database=":memory:")

# Iterate over all Parquet files in the directory
for parquet_file in parquet_file_key:
    for eval_types, filename in parquet_file.items():
        anova_parquet_file = os.path.join(anova_directory, filename)

        # Run DuckDB SQL Query directly on Parquet file
        query = f"""
        SELECT *
        FROM read_parquet('{anova_parquet_file}')
        """
        
        # Execute query
        anova_result_df = con.execute(query).fetchdf()

        print(f"\nAnova Results in {anova_parquet_file}:\n")
        print(anova_result_df.to_string(index=False))


# Close DuckDB connection
con.close()


Anova Results in dataset/eda/anova/Bias_combined_anova_results_20250308_155958_v3.parquet:

                               kpi        feature    F_value       p_value      sum_sq  df            datetime                                                                                                                    notes            factor
               bias_presence_score judge_model_id 128.174336  7.178740e-56   59.072734 2.0 2025-03-08_15-59-58                The effect of 'C(judge_model_id)' is highly statistically significant (p < 0.01) for bias_presence_score. C(judge_model_id)
               bias_presence_score  eval_model_id 277.151864 2.224164e-229  240.607769 4.0 2025-03-08_15-59-58                 The effect of 'C(eval_model_id)' is highly statistically significant (p < 0.01) for bias_presence_score.  C(eval_model_id)
            is_gender_bias_present judge_model_id  13.801445  1.028543e-06    1.204144 2.0 2025-03-08_15-59-58             The effect of 'C(judge_model_id)' 

In [9]:

# Set the significance threshold (p-value >= 0.05 is non-significant)
p_value_threshold = 0.05

# Initialize DuckDB connection (in-memory)
con = duckdb.connect(database=":memory:")

# Iterate over all Parquet files in the directory
for parquet_file in parquet_file_key:
    for eval_types, filename in parquet_file.items():
        anova_parquet_file = os.path.join(anova_directory, filename)

        # Run DuckDB SQL Query directly on Parquet file
        query = f"""
        SELECT kpi, feature, factor, p_value
        FROM read_parquet('{anova_parquet_file}')
        WHERE p_value >= {p_value_threshold}
        """

        # Execute query
        non_significant_results = con.execute(query).fetchdf()  # Returns a Pandas DataFrame

        # Print only if there are non-significant results
        if not non_significant_results.empty:
            print(f"\nAnova Non-Significant Results in {anova_parquet_file}:\n")
            print(non_significant_results.to_string(index=False))
        else:
            print(f"\nAnova Results in {anova_parquet_file}:\n")
            print(f"\nNo Non-Significant Results Found\n")

# Close DuckDB connection
con.close()



Anova Results in dataset/eda/anova/Bias_combined_anova_results_20250308_155958_v3.parquet:


No Non-Significant Results Found


Anova Results in dataset/eda/anova/Ethics_combined_anova_results_20250308_160000_v3.parquet:


No Non-Significant Results Found


Anova Results in dataset/eda/anova/Fairness_combined_anova_results_20250308_155959_v3.parquet:


No Non-Significant Results Found


Anova Results in dataset/eda/anova/Factuality_combined_anova_results_20250308_160000_v3.parquet:


No Non-Significant Results Found



## Beats Eval results dataset:

In [10]:
beats_parquet_file_name = "beats_eval_dataset_20250514_175840.parquet"
beats_eval_directory = "dataset/beats_eval/eval_dataset/"

In [12]:
beats_eval_parquet_file = os.path.join(
    beats_eval_directory, beats_parquet_file_name
)
print(beats_eval_parquet_file)

dataset/beats_eval/eval_dataset/beats_eval_dataset_20250514_175840.parquet


In [13]:
def execute_sql_query(sql_query):
    try:
        # Print the result in a pretty table format
        pd.set_option('display.float_format', '{:.2f}'.format)
        sql_query_result = duckdb.query(sql_query).to_df()
        display(sql_query_result)

    except duckdb.Error as e:
        error_message = f"An error occurred while executing the query: {e}"
        print(error_message)
        return False, error_message

In [14]:
query_factuality_score_analysis = f"""
SELECT
    CASE
        WHEN factual_accuracy_score BETWEEN 1 AND 3 THEN 'Low'
        WHEN factual_accuracy_score BETWEEN 4 AND 6 THEN 'Mid'
        WHEN factual_accuracy_score BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS factual_accuracy_range,
    CASE
        WHEN misinformation_risk_score BETWEEN 1 AND 3 THEN 'Low'
        WHEN misinformation_risk_score BETWEEN 4 AND 6 THEN 'Mid'
        WHEN misinformation_risk_score BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS misinformation_risk_range,
    COUNT(*) AS number_of_records
FROM
    read_parquet('{beats_eval_parquet_file}')
GROUP BY
    factual_accuracy_range,
    misinformation_risk_range
ORDER BY
    CASE factual_accuracy_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END,
    CASE misinformation_risk_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END;
"""

In [15]:
query_arg = query_factuality_score_analysis

In [16]:
execute_sql_query(query_arg)

,factual_accuracy_range,misinformation_risk_range,number_of_records
0,Low,Low,37
1,Low,Mid,236
2,Low,High,359
3,Mid,Low,60
4,Mid,Mid,1463
5,Mid,High,292
6,High,Low,10024
7,High,Mid,1034
8,High,High,10


In [20]:
query_fairness_score_analysis = f"""SELECT
    CASE
        WHEN demographic_parity_score BETWEEN 1 AND 3 THEN 'Low'
        WHEN demographic_parity_score BETWEEN 4 AND 6 THEN 'Mid'
        WHEN demographic_parity_score BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS demographic_parity_range,
    CASE
        WHEN equal_opportunity_score BETWEEN 1 AND 3 THEN 'Low'
        WHEN equal_opportunity_score BETWEEN 4 AND 6 THEN 'Mid'
        WHEN equal_opportunity_score BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS equal_opportunity_range,
    CASE
        WHEN group_fairness_index  BETWEEN 1 AND 3 THEN 'Low'
        WHEN group_fairness_index  BETWEEN 4 AND 6 THEN 'Mid'
        WHEN group_fairness_index  BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS group_fairness_range,
    COUNT(*) AS number_of_records
FROM
    read_parquet('{beats_eval_parquet_file}')
GROUP BY
    demographic_parity_range,
    equal_opportunity_range,
    group_fairness_range
ORDER BY
    CASE demographic_parity_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END,
    CASE equal_opportunity_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END,
        CASE group_fairness_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END;"""

In [21]:
query_arg = query_fairness_score_analysis

In [22]:
execute_sql_query(query_arg)

,demographic_parity_range,equal_opportunity_range,group_fairness_range,number_of_records
0,Low,Low,Low,388
1,Low,Low,Mid,9
2,Low,Low,High,1
3,Low,Mid,Low,187
4,Low,Mid,Mid,125
5,Mid,Low,Low,38
6,Mid,Low,Mid,103
7,Mid,Mid,Low,68
8,Mid,Mid,Mid,3038
9,Mid,Mid,High,10


In [23]:
query_ethics_score_analysis = f"""SELECT
    CASE
        WHEN ethical_alignment_index BETWEEN 1 AND 3 THEN 'Low'
        WHEN ethical_alignment_index BETWEEN 4 AND 6 THEN 'Mid'
        WHEN ethical_alignment_index BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS ethical_alignment_range,
    CASE
        WHEN value_alignment_score BETWEEN 1 AND 3 THEN 'Low'
        WHEN value_alignment_score BETWEEN 4 AND 6 THEN 'Mid'
        WHEN value_alignment_score BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS value_alignment_range,
    CASE
        WHEN harm_prevention_score  BETWEEN 1 AND 3 THEN 'Low'
        WHEN harm_prevention_score  BETWEEN 4 AND 6 THEN 'Mid'
        WHEN harm_prevention_score  BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS harm_prevention_range,
    CASE
        WHEN cultural_sensitivity_score BETWEEN 1 AND 3 THEN 'Low'
        WHEN cultural_sensitivity_score BETWEEN 4 AND 6 THEN 'Mid'
        WHEN cultural_sensitivity_score BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS cultural_sensitivity_range,
    CASE
        WHEN inclusivity_score BETWEEN 1 AND 3 THEN 'Low'
        WHEN inclusivity_score BETWEEN 4 AND 6 THEN 'Mid'
        WHEN inclusivity_score BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS inclusivity_range,
    COUNT(*) AS number_of_records
FROM
    read_parquet('{beats_eval_parquet_file}')
GROUP BY
    ethical_alignment_range,
    value_alignment_range,
    harm_prevention_range,
    cultural_sensitivity_range,
    inclusivity_range
ORDER BY
    CASE ethical_alignment_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END,
    CASE value_alignment_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END,
        CASE harm_prevention_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END,
    CASE cultural_sensitivity_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END,
        CASE inclusivity_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END;"""

In [24]:
query_arg = query_ethics_score_analysis

In [25]:
execute_sql_query(query_arg)

,ethical_alignment_range,value_alignment_range,harm_prevention_range,cultural_sensitivity_range,inclusivity_range,number_of_records
0,Low,Low,Low,Low,Low,267
1,Low,Low,Low,Low,Mid,2
2,Low,Low,Low,Mid,Low,37
3,Low,Low,Low,Mid,Mid,4
4,Low,Low,Mid,Low,Low,14
...,...,...,...,...,...,...
72,High,High,High,Low,Mid,1
73,High,High,High,Mid,Mid,113
74,High,High,High,Mid,High,46
75,High,High,High,High,Mid,110


In [26]:
query_bias_presence_analysis = f"""select
    eval_model_id,
    sum(bias_presence_score) as total_bias_presence_score,
    sum(is_gender_bias_present) as total_is_gender_bias_present,
    sum(is_race_and_ethnicity_bias_present) as total_is_race_and_ethnicity_bias_present,
    sum(is_socioeconomic_bias_present) as total_is_socioeconomic_bias_present,
    sum(is_cultural_bias_present) as total_is_cultural_bias_present,
    sum(is_religion_bias_present) as total_is_religion_bias_present,
    sum(is_sexual_orientation_bias_present) as total_is_sexual_orientation_bias_present,
    sum(is_disability_bias_present) as total_is_disability_bias_present,
    sum(is_age_bias_present) as total_is_age_bias_present,
    sum(is_geography_bias_present) as total_is_geography_bias_present,
    sum(is_political_bias_present) as total_is_political_bias_present,
    sum(is_stereotype_bias_present) as total_is_stereotype_bias_present,
    sum(intersectional_bias) as total_intersectional_bias
from
    read_parquet('{beats_eval_parquet_file}')
group by
    eval_model_id;"""

In [27]:
query_arg = query_bias_presence_analysis

In [28]:
execute_sql_query(query_arg)

,eval_model_id,total_bias_presence_score,total_is_gender_bias_present,total_is_race_and_ethnicity_bias_present,total_is_socioeconomic_bias_present,total_is_cultural_bias_present,total_is_religion_bias_present,total_is_sexual_orientation_bias_present,total_is_disability_bias_present,total_is_age_bias_present,total_is_geography_bias_present,total_is_political_bias_present,total_is_stereotype_bias_present,total_intersectional_bias
0,claude-3-5-sonnet-20241022,649.00,72.00,189.00,222.00,286.00,49.00,12.00,40.00,110.00,136.00,40.00,517.00,213.00
1,mistral-large-latest,1151.00,118.00,392.00,412.00,551.00,87.00,47.00,113.00,219.00,265.00,124.00,961.00,415.00
2,meta.llama3-1-405b-instruct-v1:0,1658.00,282.00,517.00,532.00,775.00,103.00,57.00,142.00,258.00,366.00,145.00,1393.00,536.00
3,gpt-4o-2024-08-06,865.00,66.00,269.00,319.00,408.00,48.00,19.00,69.00,162.00,230.00,74.00,705.00,312.00
4,gemini-1.5-pro-002,766.00,81.00,247.00,273.00,312.00,62.00,18.00,45.00,141.00,132.00,62.00,631.00,266.00


In [29]:
query_bias_magnitude_score_analysis = f"""SELECT
    CASE
        WHEN bias_severity_score BETWEEN 1 AND 3 THEN 'Low'
        WHEN bias_severity_score BETWEEN 4 AND 6 THEN 'Mid'
        WHEN bias_severity_score BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS severity_range,
    CASE
        WHEN bias_impact_score BETWEEN 1 AND 3 THEN 'Low'
        WHEN bias_impact_score BETWEEN 4 AND 6 THEN 'Mid'
        WHEN bias_impact_score BETWEEN 7 AND 10 THEN 'High'
        ELSE 'Unknown'
    END AS impact_range,
    COUNT(*) AS number_of_records
FROM
    read_parquet('{beats_eval_parquet_file}')
GROUP BY
    severity_range,
    impact_range
ORDER BY
    CASE severity_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END,
    CASE impact_range
        WHEN 'Low' THEN 1
        WHEN 'Mid' THEN 2
        WHEN 'High' THEN 3
        ELSE 4
    END;"""

In [30]:
query_arg = query_bias_magnitude_score_analysis

In [31]:
execute_sql_query(query_arg)

,severity_range,impact_range,number_of_records
0,Low,Low,8960
1,Low,Mid,185
2,Low,High,1
3,Mid,Low,473
4,Mid,Mid,2480
5,Mid,High,340
6,High,Low,23
7,High,Mid,214
8,High,High,839
